In [1]:
import pandas as pd
import geopandas as gpd
import zipfile
import numpy as np
import plotly.express as px
import shapely.geometry as sgeo

In [2]:
df_zone = gpd.read_file("../../analysis/data/mvv_area.gpkg")

In [3]:
df_geo = pd.read_csv("../../output/zones/eqasim_legs.csv", sep = ";", usecols = [
    "person_id", "person_trip_id", "leg_index",
    "origin_x", "origin_y", "destination_x", "destination_y"
])
origin = gpd.points_from_xy(df_geo["origin_x"], df_geo["origin_y"])
destination = gpd.points_from_xy(df_geo["destination_x"], df_geo["destination_y"])
df_geo["geometry"] = [sgeo.LineString(od) for od in zip(origin, destination)]
df_geo = gpd.GeoDataFrame(df_geo, crs = df_zone.crs)
df_geo = gpd.sjoin(df_geo, df_zone, predicate = "within")[["person_id", "person_trip_id", "leg_index"]].copy()

In [4]:
df = pd.read_csv("../../output/zones/eqasim_pt.csv", sep = ";")
df = pd.merge(df, df_geo, on = ["person_id", "person_trip_id", "leg_index"], how = "inner")

In [5]:
df = df.groupby("transit_mode").size().reset_index(name = "count")
df["share"] = df["count"] / df["count"].sum()
df["source"] = "simulation"

In [6]:
df_reference = pd.DataFrame({
    "transit_mode": [
        "bus", "tram", "subway"
    ],
    "share": [
        0.268, 0.134, 0.599, 
    ]
})

df_reference["source"] = "mvg"

In [7]:
df = pd.concat([df, df_reference])

In [8]:
px.bar(
    df, x = "transit_mode", y = "share", color = "source", barmode = "group",
    title = "Comparison all simulation modes"
)

In [9]:
df = df[df["transit_mode"].isin(df_reference["transit_mode"])]
df.loc[df["source"] == "simulation", "share"] = df.loc[df["source"] == "simulation", "count"] / df.loc[df["source"] == "simulation", "count"].sum()

In [10]:
px.bar(
    df, x = "transit_mode", y = "share", color = "source", barmode = "group",
    title = "Comparison MVG modes"
)